In [8]:
from typing import DefaultDict
from manim import *
import subprocess, sys
import numpy as np
import math

config.media_embed = True

from collections import defaultdict

#############################################
# Generalized N pendulum
#############################################
class Pendulum:
    def __init__(self, N, theta=None):
        self.N = N;
        self.gravity = 9.81
        self.L = np.ones(shape=(N, )) # lengths
        self.m = np.ones(shape=(N, )) # masses
        
        self.theta = np.zeros(shape=(N, )) if theta is None else theta
        self.theta_dot = np.zeros(shape=(N, ))

    def step(self, delta_t):
        equations = np.zeros(shape=(4 * self.N, 4 * self.N + 1))
        theta_dd = 0
        a_xn = theta_dd + self.N
        a_yn = a_xn + self.N
        T_n = a_yn + self.N
        for p in range(0, 4 * self.N, 4):
            n = p // 4
            # First equation x_dd
            equations[p, theta_dd + n] = self.L[n] * math.cos(self.theta[n])
            equations[p, a_xn + n] = -1.0;
            if n > 0:
                equations[p, a_xn + n - 1] = 1
            equations[p, -1] = self.theta_dot[n]**2 * math.sin(self.theta[n])

            # Second equation y_dd
            equations[p + 1, theta_dd + n] = -1.0 * self.L[n] * math.sin(self.theta[n])
            equations[p + 1, a_yn + n] = -1.0
            if n > 0:
                equations[p + 1, a_yn + n - 1] = 1.0
            equations[p + 1, -1] = self.theta_dot[n]**2 * math.cos(self.theta[n])

            # Third equation horizontal tension
            equations[p + 2, a_xn + n] = self.m[n]
            equations[p + 2, T_n + n] = math.sin(self.theta[n])
            if n + 1 < self.N:
                equations[p + 2, T_n + n + 1] = -1.0 * math.sin(self.theta[n + 1])
            equations[p + 2, -1] = 0

            # Fourth equation vertical tension
            equations[p + 3, a_yn + n] = self.m[n]
            equations[p + 3, T_n + n] = math.cos(self.theta[n])
            if n + 1 < self.N:
                equations[p + 3, T_n + n + 1] = -1.0 * math.cos(self.theta[n + 1])
            equations[p + 3, -1] = self.m[n] * self.gravity

        solution = np.linalg.solve(equations[:, 0:-1], equations[:, -1])
        self.theta_dot += delta_t * solution[0:self.N]
        self.theta += delta_t * self.theta_dot


class PhysicalNPendulum(Scene):
    def construct(self):
        p = Pendulum(N=2, theta=[math.pi / 2] * 2)
        scale = 2
        x_c, y_c = 0, 3

        def get_pendulum(dot_radius=DEFAULT_DOT_RADIUS):
            points = [[x_c, y_c, 0]] + [[scale * p.L[n] * math.sin(p.theta[n]), -scale * p.L[n] * math.cos(p.theta[n]), 0] for n in range(p.N)]
            for i in range(1, len(points)):
                for k in range(3):
                    points[i][k] += points[i - 1][k]
            return [ Line(points[i - 1], points[i]) for i in range(1, len(points)) ] + [ Dot(point, radius=dot_radius) for point in points]

        def step(pendulum, dt):
            for obj in pendulum:
                pendulum.remove(obj)
            p.step(dt / 2)
            for obj in get_pendulum(dot_radius=0.06):
                pendulum.add(obj)

        pendulum = VGroup()
        for obj in get_pendulum():
            pendulum.add(obj)
        pendulum.add_updater(step)
        self.add(pendulum)
        self.wait(5)
        pendulum.remove_updater(step)


param= "-v WARNING  --progress_bar None   -r  500,200 --fps=3 --disable_caching PhysicalNPendulum"
paramgif= "-v WARNING  --progress_bar None --format=gif  -r  500,200  --disable_caching PhysicalNPendulum"
%manim $param 